# Static Load Calculations for Snow Slabs

This notebook demonstrates methods for calculating the static load (weight per unit area) of layered snow slabs and their force components on slopes using the `snowpyt_mechparams` package.

## Overview

The static load calculations include:
- **Gravitational load**: Total weight per unit area (vertical force)
- **Shear load**: Force component parallel to the slope surface
- **Normal load**: Force component perpendicular to the slope surface

These calculations are essential for avalanche hazard assessment and snow stability analysis.


In [10]:
# Import Libraries
import os
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpyt_mechparams.data_structures import Layer, Slab
from snowpyt_mechparams.static_load import calculate_static_load
from snowpyt_mechparams import density


## Parse Snowpit Files

First, let's parse the CAAML snowpit files to extract real layer data:


In [11]:
all_pits = []
failed_files = []

folder_path = 'data'

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for file in xml_files:
    try:
        file_path = os.path.join(folder_path, file)
        pit = caaml_parser(file_path)
        all_pits.append(pit)
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Warning: Failed to parse {file}: {e}")

print(f"Successfully parsed {len(all_pits)} files")
print(f"Failed to parse {len(failed_files)} files")


Successfully parsed 50278 files
Failed to parse 0 files


## Extract Layer Data and Calculate Densities

Now let's extract layer information and calculate densities using the Geldsetzer method:


In [12]:
def convert_grain_form(grain_form_obj):
    """
    Convert grain form object to code needed for Geldsetzer table.
    
    Parameters:
    grain_form_obj: Grain form object from CAAML data
    
    Returns:
    str: Grain form code for Geldsetzer table lookup, or None if not mappable
    """
    # Handle None grain form objects
    if grain_form_obj is None:
        return None
    
    # Check if sub_grain_class_code exists and is in our target codes
    if (hasattr(grain_form_obj, 'sub_grain_class_code') and 
        grain_form_obj.sub_grain_class_code and
        grain_form_obj.sub_grain_class_code in ["PPgp", "RGmx", "FCmx"]):
        return grain_form_obj.sub_grain_class_code
    
    # Otherwise, return basic grain class code if available
    if hasattr(grain_form_obj, 'basic_grain_class_code') and grain_form_obj.basic_grain_class_code:
        return grain_form_obj.basic_grain_class_code
    
    return None


def calculate_layer_density(hand_hardness, grain_form):
    """
    Calculate density using Geldsetzer method.
    
    Parameters:
    hand_hardness: Hand hardness value
    grain_form: Grain form code
    
    Returns:
    float: Density in kg/m³, or None if calculation fails
    """
    try:
        if pd.isna(grain_form) or grain_form is None:
            return None
        
        return density.calculate_density(
            method='geldsetzer',
            hand_hardness=hand_hardness,
            grain_form=grain_form
        )
    except (ValueError, TypeError):
        return None


# Collect relevant data from each snowpit
pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)
    
    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'depth_top': layer.depth_top,
            'thickness': layer.thickness,
            'hand_hardness': layer.hardness,
            'wetness': layer.wetness,
            'layer_of_concern': layer.layer_of_concern,
            'grain_form_primary': layer.grain_form_primary,
            'grain_form_secondary': layer.grain_form_secondary,
        }
        
        # Add geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['geldsetzer_grain_form'] = convert_grain_form(layer.grain_form_primary)
        else:
            layer_dict['geldsetzer_grain_form'] = None
            
        # Calculate density if we have the required data
        if layer.hardness is not None and layer_dict['geldsetzer_grain_form'] is not None:
            layer_dict['calculated_density'] = calculate_layer_density(
                layer.hardness, 
                layer_dict['geldsetzer_grain_form']
            )
        else:
            layer_dict['calculated_density'] = None
        
        layer_info.append(layer_dict)

# Create dataframes from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)

print("=== Dataset Summary ===")
print(f"Total number of snowpits: {len(all_pits)}")
print(f"Total number of layers: {len(layer_df)}")

# Filter layers with both hand hardness and grain form data
layers_with_data = layer_df[layer_df['hand_hardness'].notna() & layer_df['grain_form_primary'].notna()]
print(f"Layers with both hand hardness and grain form data: {len(layers_with_data)}")
print(f"Percentage of layers with complete data: {len(layers_with_data)/len(layer_df)*100:.1f}%")

# Filter layers with calculated density (usable for static load calculations)
layers_with_density = layer_df[layer_df['calculated_density'].notna()]
print(f"Layers with calculated density: {len(layers_with_density)}")
print(f"Percentage of layers with density: {len(layers_with_density)/len(layer_df)*100:.1f}%")


=== Dataset Summary ===
Total number of snowpits: 50278
Total number of layers: 371429
Layers with both hand hardness and grain form data: 279331
Percentage of layers with complete data: 75.2%
Layers with calculated density: 199344
Percentage of layers with density: 53.7%


## Create Snow Slabs from Real Data

Now let's convert the parsed snowpit data into `Layer` and `Slab` objects for static load calculations:


In [14]:
def create_slab_from_pit(pit_id, layer_df):
    """
    Create a Slab object from layers of a specific snowpit.
    
    Parameters:
    pit_id: ID of the snowpit
    layer_df: DataFrame containing layer information
    
    Returns:
    Slab: Slab object with layers that have calculated densities, or None if no valid layers
    """
    # Get layers for this pit that have calculated densities
    pit_layers = layer_df[
        (layer_df['pit_id'] == pit_id) & 
        (layer_df['calculated_density'].notna()) &
        (layer_df['thickness'].notna()) &
        (layer_df['thickness'] > 0)
    ].copy()
    
    if len(pit_layers) == 0:
        return None
    
    # Sort layers by depth (top to bottom)
    pit_layers = pit_layers.sort_values('depth_top')
    
    # Create Layer objects
    layers = []
    for _, row in pit_layers.iterrows():
        # Convert thickness from cm to mm (if needed)
        thickness_mm = row['thickness'] * 10 if row['thickness'] < 100 else row['thickness']
        
        layer = Layer(
            thickness=thickness_mm,
            density=row['calculated_density']
        )
        layers.append(layer)
    
    if len(layers) == 0:
        return None
        
    return Slab(layers=layers)


## Static Load Calculations

